# Real Data Simulation

@Author: Daniel Rodríguez Delgado

@Date: 02/09/2019

In [6]:
from alpha_vantage.timeseries import TimeSeries
ts=TimeSeries(key='FT8ZMTO1NJSILIKB', output_format='pandas')
import time
import pandas as pd
import numpy as np
from math import *
from scipy.stats import kurtosis, skew, pearsonr, norm, t
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


## 1. Pseudo-random number generators

In [7]:
def congru(seed, m=((2**31)-1), a=16807, b=0):
        y=(a*seed+b)%m
        u=y/m
        return(u, y)
    
def boxmuller(seed, mu=0, sigma=1):
    u1, seed=congru(seed)    #Uniform inputs
    u2, seed=congru(seed)
    
    x=sqrt(-2*log(u1))*cos(2*pi*u2)*sigma+mu  #Gaussian numbers generation
    y=sqrt(-2*log(u1))*sin(2*pi*u2)*sigma+mu
    
    antx=sqrt(-2*log(1-u1))*cos(2*pi*(1-u2))*sigma+mu  #Antithetic numbers generation
    anty=sqrt(-2*log(1-u1))*sin(2*pi*(1-u2))*sigma+mu
    return(x, y, u1, u2, antx, anty, seed)


def marsaglia(seed, mu=0, sigma=1, mode='normal'):
    
    while True:
        if mode=='normal':
            u1, seed= congru(seed)
            u2, seed= congru(seed)
            v1=2*u1-1
            v2=2*u2-1
            R2=v1**2+v2**2
            if R2<=1:
                y=sqrt(-2*log(R2)/R2)
                x1=v1*y*sigma+mu
                x2=v2*y*sigma+mu
                break
        if mode=='antithetic':
            u1, seed= congru(seed)
            u2, seed= congru(seed)
            v1=2*(1-u1)-1
            v2=2*(1-u2)-1
            R2=v1**2+v2**2
            if R2<=1:
                y=sqrt(-2*log(R2)/R2)
                x1=v1*y*sigma+mu
                x2=v2*y*sigma+mu
                break
        
    return(x1,x2,seed)

# Gamma distribution random generator
def gamma (p, a, seed):
    v=[]
    for i in range(p):
        u, seed= congru(seed)
        v.append(log(u))
    x=-sum(v)/a
    return(x, seed)


# Chi^2 distribution random generator
def chisq (n, seed):
    if (n%2==0):
        g, seed=gamma(int(n/2), 1/2, seed)
        return(g, seed)
    else:
        g, seed=gamma(int((n-1)/2), 1/2, seed)
        cos1, sen1, u1, u2, antx, anty, seed = boxmuller(seed)
        x=g+sen1**2
    return(x, seed)

# t-student distribution random generator
def tstudent (n, seed):
    cos1, sen1, u1, u2, antx, anty, seed=boxmuller(seed)
    xchi, seed=chisq(n, seed)
    x=sen1/sqrt(xchi/n)
    return(x, seed)


## 2. Data load

In [12]:
import time


tickers=['AAPL','AMZN','ALXN','NFLX','EXPE','PYPL','TSLA','FOX']

companies=pd.DataFrame()
dividends=pd.DataFrame()
data=pd.DataFrame()
meta_data=pd.DataFrame()

for i in range(len(tickers)):
    ticker=tickers[i]
    print(ticker)
    data, meta_data = ts.get_daily_adjusted(symbol=ticker,outputsize='full')
    
    divs=pd.DataFrame(data['7. dividend amount'])
    data=pd.DataFrame(data['5. adjusted close'])            
    
    data=data.loc['2018-08-31':'2019-01-25']
    divs=divs.loc['2018-01-02':'2018-12-31']
    
    dividends=pd.concat([dividends, divs], axis=1,sort=False)
    companies=pd.concat([companies, data], axis=1,sort=False)
    
    time.sleep(5)
    
companies.columns = tickers
dividends.columns = tickers

AAPL
AMZN
ALXN
NFLX
EXPE
PYPL
TSLA


KeyError: 'Time Series (Daily)'